In [1]:
!pip install autokeras

     |████████████████████████████████| 174kB 8.3MB/s 
     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=d38c8a27d666ec21bf5a4a00b61ffc675993e34f6aeae72cf672777689646e11
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=119a9221a51ca441ee1832300f898135952384bcbc3cc8c8abc6001b461f7bf0
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
pip install clearml

     |████████████████████████████████| 993kB 7.1MB/s 
     |████████████████████████████████| 92kB 6.7MB/s 


In [3]:
import numpy as np
import tensorflow as tf
import autokeras as ak
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from pathlib import Path
#preprocess.
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os  
import cv2 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


try clearml for visualizing callbacks in training/testing model results

In [4]:
from clearml import Task
task = Task.init(project_name='automl', task_name='automl_diy_model')
from tensorflow import keras


ClearML Task: created new task id=900b080296254a84bc58ed6fda3bbb06
2021-05-10 18:00:21,164 - clearml.Task - INFO - No repository found, storing script code instead
2021-05-10 18:00:21,544 - clearml - WARNING - 'NoneType' object has no attribute 'from_config'
ClearML results page: https://demoapp.demo.clear.ml/projects/1be01d2f33074d8d89f15e62b3e4d864/experiments/900b080296254a84bc58ed6fda3bbb06/output/log


# Data Interpretation

This dataset contains 853 images belonging to the 3 classes, as well as their bounding boxes in the PASCAL VOC format.
The classes are:

With mask;
Without mask;
Mask worn incorrectly.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/My Drive/EECS738/annotations'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Data prep
* label encoding

* train/test split

In [6]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# print(x_train.shape)  # (60000, 28, 28)
# print(y_train.shape)  # (60000,)
# print(y_train[:3])  # array([7, 2, 1], dtype=uint8)
# Functions to load dataset
def load_image(img_file):
    """
    Loads image file, resize to 128 x 128 and return as numpy array
    :param img_file: path to image file in .jpg format
    :return: numpy array (128, 128, 3)
    """
    img = Image.open(img_file)
    img = img.resize((128, 128))
    img = np.array(img)
    return img

def load_dataset(data_dir, task='classification'):
    """
    Loads x and y dataset for classification or regression
    :param data_dir: path to data folder containing images and CSV file
    :param task: 'classification' or 'regression', default is 'classification'
    :return: x and y dataset
    """
    # loads x data, images sorted according to plot number
    img_list = [load_image(file) for file in sorted(Path(data_dir).glob('*.jpg'),key=lambda x: int(x.stem.split('_')[1]))]
    x = np.stack(img_list)
    # loads y data
    csv_file = Path(data_dir) / 'Lodging_data.csv'
    df = pd.read_csv(csv_file)
    if task == 'regression':
        df = df[df['Lodging'] == 'Yes']  # select rows for only lodged plots
        y = df['Score']
        x = [x[i] for i in df.index]  # slice x for only lodged plots
        x = np.array(x)
    else:
        y = df['Lodging']
    return x, y    

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
def assign_label(img,flower_type):
    return flower_type

X=[]
Z=[]
IMG_SIZE=150
FLOWER_DAISY_DIR='/content/drive/My Drive/EECS738/daisy/'
FLOWER_SUNFLOWER_DIR='/content/drive/My Drive/EECS738/sunflower/'
FLOWER_TULIP_DIR='/content/drive/My Drive/EECS738/tulip'
FLOWER_DANDI_DIR='/content/drive/My Drive/EECS738/dandelion'
FLOWER_ROSE_DIR='/content/drive/My Drive/EECS738/rose'
def assign_label(img,flower_type):
    return flower_type

def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))    

In [7]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
make_train_data('Tulip',FLOWER_TULIP_DIR)
make_train_data('Dandelion',FLOWER_DANDI_DIR)
make_train_data('Rose',FLOWER_ROSE_DIR)

100%|██████████| 784/784 [05:46<00:00,  2.26it/s]


In [ ]:
 # %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [9]:
# debug block
print(x_train.shape)
print(y_train.shape)
print(X.shape)
# print(x_train[:1])
# Z

(2896, 150, 150, 3)
(2896, 5)
(4323, 150, 150, 3)


In [8]:
from keras.utils import to_categorical
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)
X=np.array(X)
# X=X/255
# split train/test
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.33,random_state=42)


apply autokeras classification

In [13]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10, callbacks=[tensorboard_callback_train])


# Predict with the best model.
predicted_yy = clf.predict(x_test)
print(predicted_yy)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 1 Complete [00h 02m 01s]
val_loss: 1.0518347024917603

Best val_loss So Far: 1.0518347024917603
Total elapsed time: 00h 02m 01s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
91/91 [==============================] - 9s 91ms/step - loss: 3.7835 - accuracy: 0.3047
Epoch 2/10
91/91 [==============================] - 8s 86ms/step - loss: 1.0392 - accuracy: 0.5888
Epoch 3/10
91/91 [==============================] - 8s 87ms/step - loss: 0.7321 - accuracy: 0.7422
Epoch 4/10
91/91 [==============================] - 8s 86ms/step - loss: 0.3908 - accuracy: 0.8734
Epoch 5/10
91/91 [==============================] - 8s 87ms/step - loss: 0.2676 - accuracy: 0.9223
Epoch 6/10
91/91 [==============================] - 8s 86ms/step - loss: 0.1982 - accuracy: 0.9439
Epoch 7/10
91/91 [==============================] - 8s 86ms/step - loss: 0.1076 - accuracy: 0.9709
Epoch 8/10
91/91 [==============================] - 8s 87ms/step - loss: 0.1056 - accuracy: 0.9702
Epoch 9/10
91/91 [==================

explore classification models in autokeras

3classic models to look at:

CNN, ResNet, Xception

In [10]:

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock()(output_node)
output_node3 = ak.XceptionBlock()(output_node)
# output_node3 = ak.DenseBlock()(output_node)
# output_node4 = ak.Embedding()(output_node)

output_node = ak.Merge()([output_node1, output_node2, output_node3])
output_node = ak.ClassificationHead()(output_node)

auto_model = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

# clf = autokeras.ImageClassifier(
#     num_classes=None,
#     multi_label=False,
#     loss=None,
#     metrics=None,
#     project_name="image_classifier",
#     max_trials=100,
#     directory=None,
#     objective="val_loss",
#     tuner=None,
#     overwrite=False,
#     seed=42,
#     max_model_size=None,
#     **kwargs
# )
tensorboard_callback_train = keras.callbacks.TensorBoard(log_dir='log',histogram_freq=1,
  write_images=True)
# Feed the AutoModel with training data.
auto_model.fit(x_train[:], y_train[:], epochs=10, callbacks=[tensorboard_callback_train])
# Predict with the best model.
predicted_y = auto_model.predict(x_test)

# Evaluate the best model with testing data.
print(auto_model.evaluate(x_test, y_test))
# get the best performing model
model = auto_model.export_model()
# summarize the loaded model
model.summary()
# save the best performing model to file
model.save('model_clf.h5')
# save results summary
auto_model.tuner.results_summary()

Trial 1 Complete [00h 15m 20s]
val_loss: 1.629306435585022

Best val_loss So Far: 1.629306435585022
Total elapsed time: 00h 15m 20s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
91/91 [==============================] - 104s 1s/step - loss: 12.9604 - accuracy: 0.3075
Epoch 2/10
91/91 [==============================] - 90s 985ms/step - loss: 4.1819 - accuracy: 0.4953
Epoch 3/10
91/91 [==============================] - 90s 985ms/step - loss: 4.2411 - accuracy: 0.5362
Epoch 4/10
91/91 [==============================] - 90s 984ms/step - loss: 5.6401 - accuracy: 0.5992
Epoch 5/10
91/91 [==============================] - 90s 984ms/step - loss: 1.4964 - accuracy: 0.6557
Epoch 6/10
91/91 [==============================] - 90s 985ms/step - loss: 1.1682 - accuracy: 0.7511
Epoch 7/10
91/91 [==============================] - 90s 985ms/step - loss: 0.9161 - accuracy: 0.7869
Epoch 8/10
91/91 [==============================] - 90s 985ms/step - loss: 0.5636 - accuracy: 0.8487
Epoch 9/10
91/91 [=====

In [ ]:

# get the best performing model
model = auto_model.export_model()
# summarize the loaded model
model.summary()
# save the best performing model to file
model.save('model_clf.h5')
# save results summary
auto_model.tuner.results_summary()

NameError: ignored

Customized Search Space for autokeras 

In [ ]:
# model = ak.AutoModel(tuner='bayesian',
#                      inputs=input_node,
#                      outputs=output_node,
#                      max_trials=100,
#                      overwrite=True,
#                      seed=10)

# # Train model
# model.fit(x_train, y_train, epochs=200,
#           validation_data=(x_val, y_val))    

# Evaluate model
# score = model.evaluate(x_test, y_test)

input_node = ak.ImageInput()
output_node = ak.ImageBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=100)

Trial 1 Complete [00h 14m 53s]
val_loss: 0.4090674817562103

Best val_loss So Far: 0.4090674817562103
Total elapsed time: 00h 14m 53s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
36/36 [==============================] - 100s 3s/step - loss: 0.8529 - accuracy: 0.4892
Epoch 2/10
36/36 [==============================] - 97s 3s/step - loss: 0.8187 - accuracy: 0.5911
Epoch 3/10
36/36 [==============================] - 97s 3s/step - loss: 0.5810 - accuracy: 0.6614
Epoch 4/10
36/36 [==============================] - 97s 3s/step - loss: 0.5181 - accuracy: 0.7177
Epoch 5/10
36/36 [==============================] - 96s 3s/step - loss: 0.4861 - accuracy: 0.7423
Epoch 6/10
36/36 [==============================] - 96s 3s/step - loss: 0.4591 - accuracy: 0.7592
Epoch 7/10
36/36 [==============================] - 96s 3s/step - loss: 0.4379 - accuracy: 0.7727
Epoch 8/10
36/36 [==============================] - 98s 3s/step - loss: 0.4205 - accuracy: 0.7879
Epoch 9/10
36/36 [=========================

In [ ]:
# Evaluate model
score = clf.evaluate(x_test, y_test)

Visualize autokeras model

In [ ]:
# aborted, check clearml webUI for visualization
from trains import Task

task = Task.init(project_name="autokeras", task_name="autokeras imdb example with scalars")

Try DIY automl based on sklearn model

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import balanced_accuracy_score

notes:

*   The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters.

*   sklearn.pipeline.Pipeline(steps, *, memory=None, verbose=False);steps=[List of (name, transform) tuples (implementing fit/transform) that are chained]

*   ColumnTransformer: Applies transformers to columns of an array or pandas DataFrame.
*   RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score=nan, return_train_score=False): param_distributions = optimization_grid = dict or list of dicts




In [42]:
class AutoMLClassifier:
  def __init__(self, scoring_function = 'balanced_accuracy', n_iter = 50):
    self.scoring_function = scoring_function
    self.n_iter = n_iter
  
  def fit(self,X,y):
    X_train = X
    y_train = y

    categorical_values = []

    cat_subset = X_train.select_dtypes(include = ['object','category','bool'])

    for i in range(cat_subset.shape[1]):
      categorical_values.append(list(cat_subset.iloc[:,i].dropna().unique()))

    num_pipeline = Pipeline([
                         ('cleaner',SimpleImputer()),
                         ('scaler',StandardScaler())
                         ])

    cat_pipeline = Pipeline([
                        ('cleaner',SimpleImputer(strategy = 'most_frequent')),
                        ('encoder',OneHotEncoder(sparse = False, categories=categorical_values))
    ])


    preprocessor = ColumnTransformer([
      ('numerical', num_pipeline, make_column_selector(dtype_exclude=['object','category','bool'])),
      ('categorical', cat_pipeline, make_column_selector(dtype_include=['object','category','bool']))
    ])

    model_pipeline_steps = []
    model_pipeline_steps.append(('preprocessor',preprocessor))
    model_pipeline_steps.append(('feature_selector',SelectKBest(f_classif,k='all')))
    model_pipeline_steps.append(('estimator',LogisticRegression()))
    model_pipeline = Pipeline(model_pipeline_steps)

    total_features = preprocessor.fit_transform(X_train).shape[1]

    optimization_grid = []

    # Logistic regression
    optimization_grid.append({
        'preprocessor__numerical__scaler':[RobustScaler(),StandardScaler(),MinMaxScaler()],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[LogisticRegression()]
    })

    # K-nearest neighbors
    optimization_grid.append({
        'preprocessor__numerical__scaler':[RobustScaler(),StandardScaler(),MinMaxScaler()],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[KNeighborsClassifier()],
        'estimator__weights':['uniform','distance'],
        'estimator__n_neighbors':np.arange(1,20,1)
    })

    # Random Forest
    optimization_grid.append({
        'preprocessor__numerical__scaler':[None],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[RandomForestClassifier(random_state=0)],
        'estimator__n_estimators':np.arange(5,500,10),
        'estimator__criterion':['gini','entropy']
    })


    # Gradient boosting
    optimization_grid.append({
        'preprocessor__numerical__scaler':[None],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[GradientBoostingClassifier(random_state=0)],
        'estimator__n_estimators':np.arange(5,500,10),
        'estimator__learning_rate':np.linspace(0.1,0.9,20),
    })



    # Decision tree
    optimization_grid.append({
        'preprocessor__numerical__scaler':[None],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[DecisionTreeClassifier(random_state=0)],
        'estimator__criterion':['gini','entropy']
    })

    # Linear SVM
    optimization_grid.append({
        'preprocessor__numerical__scaler':[RobustScaler(),StandardScaler(),MinMaxScaler()],
        'preprocessor__numerical__cleaner__strategy':['mean','median'],
        'feature_selector__k': list(np.arange(1,total_features,5)) + ['all'],
        'estimator':[LinearSVC(random_state = 0)],
        'estimator__C': np.arange(0.1,1,0.1),
        
    })

    search = RandomizedSearchCV(
      model_pipeline,
      optimization_grid,
      n_iter=self.n_iter,
      scoring = self.scoring_function, 
      n_jobs = -1, 
      random_state = 0, 
      verbose = 3,
      cv = 5
    )

    search.fit(X_train, y_train)
    self.best_estimator_ = search.best_estimator_
    self.best_pipeline = search.best_params_
    

  
  def predict(self,X,y = None):
    return self.best_estimator_.predict(X)

  def predict_proba(self,X,y = None):
    return self.best_estimator_.predict_proba(X)

try 

In [40]:
d = load_breast_cancer()
y = d['target']
X = pd.DataFrame(d['data'],columns = d['feature_names'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [41]:
print(X_train.shape)
print(y_train.shape)

(455, 30)
(455,)


In [47]:
# run train model
tensorboard_callback_train = keras.callbacks.TensorBoard(log_dir='log',histogram_freq=1,
  write_images=True)
model = AutoMLClassifier()
model.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   46.4s finished


Results

In [44]:
balanced_accuracy_score(y_test, model.predict(X_test))

0.9510317720275139

fit model, see which model and preprocessing parameters have been chosen

In [45]:
model.best_pipeline

{'estimator': GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.9, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=415,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=0, subsample=1.0, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False),
 'estimator__learning_rate': 0.9,
 'estimator__n_estimators': 415,
 'feature_selector__k': 21,
 'preprocessor__numerical__cleaner__strategy': 'median',
 'preprocessor__numerical__scaler': None}